In [1]:
!pip install "pyiceberg[s3fs,hive]"

Defaulting to user installation because normal site-packages is not writeable
distutils: /home/sharanya/.local/lib/python3.9/site-packages
sysconfig: /home/sharanya/.local/lib64/python3.9/site-packages
user = True
home = None
root = None
prefix = None


In [2]:
from pyiceberg.catalog import load_catalog
catalog = load_catalog(
        "docs",
    **{
            "uri": "thrift://hive-ms-trino.apps.zagaopenshift.zagaopensource.com:9888",
            "s3.endpoint": "http://minio-lb.apps.zagaopenshift.zagaopensource.com:9009",
            "py-io-impl": "pyiceberg.io.pyarrow.PyArrowFileIO",
            "s3.access-key-id": "minioAdmin",
            "s3.secret-access-key": "minio1234",
    }
)

In [15]:
namespace="ingesttrace" #enter namespace
ns=catalog.list_namespaces()
print(ns)
if((namespace,) in ns):
  print("Namespace alredy exists")
else:
  catalog.create_namespace(namespace)
  print("Namespace create suceesss")



[('apm_trace',), ('data',), ('default',), ('demo_data',), ('demo_saro',), ('demo_trace',), ('docs_check',), ('docs_logs',), ('docs_mari',), ('docs_powerleveldata',), ('docs_raja',), ('docs_sa',), ('docs_sar',), ('docs_saro',), ('docs_selva',), ('hi',), ('hlo',), ('ingesting',), ('new',), ('old',), ('poc',), ('pod',), ('pod_data',), ('power_pod',), ('powerlevelpoddata',), ('powerpod',), ('saravan',), ('saravana',), ('saro',), ('saro_data',), ('sh',), ('sha',), ('sha_trace',), ('sharanya',), ('sk',), ('soh',), ('trace_in',), ('trace_inge',), ('trace_ingest',), ('trace_ingestion',), ('trace_ingestion_data',)]
Namespace create suceesss


In [45]:
# catalog.drop_namespace("docs_trace")

In [22]:
from pyiceberg.schema import Schema
from pyiceberg.types import (
    TimestampType,
    LongType,
    IntegerType,
    FloatType,
    DoubleType,
    StringType,
    NestedField,
    StructType,
    ListType
)
from pyiceberg.partitioning import PartitionSpec, PartitionField
from pyiceberg.transforms import DayTransform
from pyiceberg.table.sorting import SortOrder, SortField
from pyiceberg.transforms import IdentityTransform

# Define the schema for the spans
schema = Schema(
    NestedField(field_id=1, name="servicename", field_type=StringType(), required=False),
    NestedField(field_id=2, name="language", field_type=StringType(), required=False),
    NestedField(field_id=3, name="tracedata", field_type=StringType(), required=False),
    NestedField(field_id=4, name="createdTime", field_type=TimestampType(), required=False),
    NestedField(field_id=5, name="traceId", field_type=StringType(), required=False),
    NestedField(field_id=6, name="startUnixNanoTime", field_type=LongType(), required=False),
    NestedField(field_id=7, name="endUnixNanoTime", field_type=LongType(), required=False),
    NestedField(field_id=8, name="duration", field_type=DoubleType(), required=False),
    NestedField(field_id=9, name="requestRoute", field_type=StringType(), required=False),
    NestedField(field_id=10, name="requestMethod", field_type=StringType(), required=False),
    NestedField(field_id=11, name="responseStatusCode", field_type=StringType(), required=False), 
)

# partition_spec = PartitionSpec(
#     PartitionField(
#         source_id=1, field_id=1000, transform=DayTransform(), name="datetime_day"
#     )
# )

sort_order = SortOrder(SortField(source_id=3, transform=IdentityTransform()))

table_name = "traces"  # Define your table identifier


table = catalog.create_table(
    identifier=(namespace, table_name),
    schema=schema,
    location="s3a://ingesttrace/traces", #mention s3 bucketname
    # partition_spec=partition_spec,
    sort_order=sort_order,
)
print(f"Table '{table_name}' created successfully.")



Table 'traces' created successfully.


In [8]:
table.schema()

Schema(NestedField(field_id=1, name='servicename', field_type=StringType(), required=False), NestedField(field_id=2, name='language', field_type=StringType(), required=False), NestedField(field_id=3, name='tracedata', field_type=StringType(), required=False), NestedField(field_id=4, name='createdTime', field_type=TimestampType(), required=False), NestedField(field_id=5, name='traceId', field_type=StringType(), required=False), NestedField(field_id=6, name='startUnixNanoTime', field_type=LongType(), required=False), NestedField(field_id=7, name='endUnixNanoTime', field_type=LongType(), required=False), NestedField(field_id=8, name='duration', field_type=FloatType(), required=False), NestedField(field_id=9, name='requestRoute', field_type=StringType(), required=False), NestedField(field_id=10, name='requestMethod', field_type=StringType(), required=False), NestedField(field_id=11, name='responseStatusCode', field_type=StringType(), required=False), schema_id=0, identifier_field_ids=[])

In [23]:
catalog.list_tables("ingesttrace")  #mention namespace

[('ingesttrace', 'traces')]

In [21]:
catalog.drop_table("ingesttrace.traces") #mention namespace and tablename